In [5]:
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st

# General Parameters
num_simulations = 1000
num_periods = 12  # Fund life in years
L = 12  # Fund life in years
B = 2   # Bow factor
CC = 100  # Total committed capital

# Growth Rate Scenarios
scenarios = {
    'pessimistic': np.full(num_periods, 0.05),
    'neutral': np.full(num_periods, 0.10),
    'optimistic': np.full(num_periods, 0.15)
}

# Macro Drivers
macro_volatility = 0.02  # 2% volatility in macro drivers

# Interest Rate Patterns
patterns = ['constant_increase', 'constant_decrease', 'constant', 'increase_then_decrease', 'decrease_then_increase']
step_options = [0.25, 0.50]  # Interest rate step options

# Function to simulate the T.A. model with variable growth rates
def simulate_TA_model(L, B, G_array, CC, num_periods):
    RC = np.linspace(1, 0, num_periods)
    PIC = np.zeros(num_periods)
    NAV = np.zeros(num_periods + 1)
    D = np.zeros(num_periods)
    C = np.zeros(num_periods)
    
    NAV[0] = 0  # Initial NAV set to zero
    
    for t in range(num_periods):
        if t > 0:
            PIC[t] = PIC[t-1] + C[t-1]
        C[t] = RC[t] * (CC - PIC[t])
        D[t] = ((t / L) ** B) * NAV[t] * (1 + G_array[t])
        NAV[t+1] = NAV[t] * (1 + G_array[t]) + C[t] - D[t]
    
    return NAV, C, D

# Function to simulate interest rate scenarios
def interest_rate_scenarios(pattern_type, start_rate, num_periods, step_options=[0.25, 0.50]):
    rates = np.zeros(num_periods)
    rates[0] = start_rate
    half_period = num_periods // 2
    
    for t in range(1, num_periods):
        step = np.random.choice(step_options)
        if pattern_type == 'constant_increase':
            rates[t] = rates[t-1] + step
        elif pattern_type == 'constant_decrease':
            rates[t] = max(0, rates[t-1] - step)
        elif pattern_type == 'constant':
            rates[t] = rates[t-1]
        elif pattern_type == 'increase_then_decrease':
            if t <= half_period:
                rates[t] = rates[t-1] + step
            else:
                rates[t] = max(0, rates[t-1] - step)
        elif pattern_type == 'decrease_then_increase':
            if t <= half_period:
                rates[t] = max(0, rates[t-1] - step)
            else:
                rates[t] = rates[t-1] + step
    return rates

# Streamlit App
st.title("Monte Carlo VaR and CVaR Analysis for T.A. Model")

# Simulation Parameters
driver_types = ['growth_rate', 'macro_drivers', 'interest_rates']
driver_selection = st.selectbox("Select Driver Type for Simulation:", driver_types)

# Simulation Function
@st.cache_data
def simulate_nav_with_confidence(L, B, CC, num_periods, num_simulations, driver_type, scenarios=None):
    all_navs = []
    
    for _ in range(num_simulations):
        if driver_type == 'growth_rate':
            scenario_key = np.random.choice(list(scenarios.keys()))
            growth_rates = scenarios[scenario_key]
        elif driver_type == 'macro_drivers':
            growth_rates = np.random.normal(0.1, 0.02, num_periods)  # Mean growth rate with macro volatility
        elif driver_type == 'interest_rates':
            pattern = np.random.choice(patterns)
            rates = interest_rate_scenarios(pattern, 2.5, num_periods, [0.25, 0.50])
            growth_rates = np.full(num_periods, 0.1) + rates / 100  # Adding interest rate effects

        # Simulate the NAV and drop the last element to match num_periods
        NAV, _, _ = simulate_TA_model(L, B, growth_rates, CC, num_periods)
        all_navs.append(NAV[:-1])  # Ensure NAV has only num_periods elements
    
    all_navs = np.array(all_navs)
    mean_nav = np.mean(all_navs, axis=0)
    lower_bound = np.percentile(all_navs, 2.5, axis=0)
    upper_bound = np.percentile(all_navs, 97.5, axis=0)
    
    return mean_nav, lower_bound, upper_bound

# Run Simulation
mean_nav, lower_bound, upper_bound = simulate_nav_with_confidence(L, B, CC, num_periods, num_simulations, driver_selection, scenarios)

# Plot Results
st.write(f"NAV Timeline with Confidence Intervals for {driver_selection.replace('_', ' ').title()}")
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(range(num_periods), mean_nav, label='Mean NAV')
ax.fill_between(range(num_periods), lower_bound, upper_bound, color='gray', alpha=0.5, label='95% Confidence Interval')
ax.set_xlabel('Periods')
ax.set_ylabel('Net Asset Value (NAV)')
ax.legend()
ax.grid(True)
st.pyplot(fig)

# VaR and CVaR Calculation
@st.cache_data
def compute_var_cvar_at_step(nav_simulations, timestep, confidence_level=0.95):
    nav_at_timestep = nav_simulations[:, timestep]
    var = np.percentile(nav_at_timestep, 100 * (1 - confidence_level))
    cvar = nav_at_timestep[nav_at_timestep <= var].mean()
    return var, cvar

# Calculate VaR and CVaR
var_half_life, cvar_half_life = compute_var_cvar_at_step(mean_nav[np.newaxis, :], num_periods // 2)
var_final, cvar_final = compute_var_cvar_at_step(mean_nav[np.newaxis, :], num_periods - 1)

# Display VaR and CVaR
st.write(f"Half-Life VaR (5%): {var_half_life}")
st.write(f"Half-Life CVaR (5%): {cvar_half_life}")
st.write(f"Final Step VaR (5%): {var_final}")
st.write(f"Final Step CVaR (5%): {cvar_final}")

2024-10-07 11:10:11.995 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 11:10:12.000 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 11:10:12.001 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 11:10:12.049 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 11:10:12.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 11:10:12.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 11:10:12.051 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 11:10:12.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar